# Metatrader 5 and Python integration

## 1 - Import requested libraries

In [1]:
# Libraries dependencies
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd

# Project files
from .classes.classes import MqlTradeResult, MqlTradeRequest, MqlAccountInfo, MqlPositionInfo
from utils.trade import Trade

## 2 - Establish MetaTrader 5 connection to a specified trading account

In [2]:
# Hedge account
# if not mt5.initialize(login=5013424139, server="MetaQuotes-Demo",password="pndvym2b"):
#     print("initialize() failed, error code =",mt5.last_error())
#     mt5.shutdown()
    
# Netting account
if not mt5.initialize(login=5013526569, server="MetaQuotes-Demo",password="h3xhhvuo"):
    print("initialize() failed, error code =",mt5.last_error())
    mt5.shutdown()

### 2.1 - Get the request connection status and parameters

In [3]:
terminal_info = mt5.terminal_info()

print(f"{terminal_info.connected = }")
print(f"{terminal_info.company = }")
print(f"{terminal_info.name = }")
print(f"{terminal_info.language = }")

terminal_info.connected = True
terminal_info.company = 'MetaQuotes Software Corp.'
terminal_info.name = 'MetaTrader 5'
terminal_info.language = 'Portuguese (Brazil)'


### 2.2 - Get account info

In [2]:
account_info = MqlAccountInfo.parse_account(mt5.account_info())

print(f"{account_info.login = }")
print(f"{account_info.name = }")
print(f"{account_info.currency = }")
print(f"{account_info.leverage = }")
print(f"{account_info.balance = }")
print(f"{account_info.profit = }")
print(f"{account_info.trade_mode.name = }")
print(f"{account_info.margin_mode.name = }")
print(f"{account_info.margin_so_mode.name = }")

TypeError: MqlAccountInfo expected mt5.AccountInfo not NoneType

## 3 - Request data

### 3.1 - Request available symbols

In [5]:
symbols=mt5.symbols_get()
symbols_name = {symbol.name for symbol in symbols}
symbols_name

{'AUDCAD',
 'AUDCHF',
 'AUDJPY',
 'AUDNZD',
 'AUDUSD',
 'AUS200',
 'CADCHF',
 'CADJPY',
 'CADMXN',
 'CHFJPY',
 'CHFMXN',
 'EURAUD',
 'EURCAD',
 'EURCHF',
 'EURCZK',
 'EURDKK',
 'EURGBP',
 'EURHKD',
 'EURHUF',
 'EURJPY',
 'EURMXN',
 'EURNOK',
 'EURNZD',
 'EURPLN',
 'EURRUB',
 'EURRUR',
 'EURSEK',
 'EURTRY',
 'EURUSD',
 'EURZAR',
 'FCHI40',
 'GBPAUD',
 'GBPCAD',
 'GBPCHF',
 'GBPJPY',
 'GBPMXN',
 'GBPNOK',
 'GBPNZD',
 'GBPPLN',
 'GBPSEK',
 'GBPSGD',
 'GBPUSD',
 'GBPZAR',
 'GDAXIm',
 'HSI50',
 'MXNJPY',
 'ND100m',
 'NI225',
 'NZDCAD',
 'NZDCHF',
 'NZDJPY',
 'NZDMXN',
 'NZDSGD',
 'NZDUSD',
 'SGDJPY',
 'SP500m',
 'SPN35',
 'STOX50',
 'UK100',
 'USDARS',
 'USDCAD',
 'USDCHF',
 'USDCLP',
 'USDCNH',
 'USDCOP',
 'USDCRE',
 'USDCZK',
 'USDDKK',
 'USDGEL',
 'USDHKD',
 'USDHUF',
 'USDJPY',
 'USDMXN',
 'USDNOK',
 'USDPLN',
 'USDRUB',
 'USDRUR',
 'USDSEK',
 'USDSGD',
 'USDTRY',
 'USDZAR',
 'XAGEUR',
 'XAGUSD',
 'XAUAUD',
 'XAUEUR',
 'XAUUSD',
 'XPDUSD',
 'XPTUSD'}

### 3.2 - Request ticks

In [6]:
# request 1000 ticks from EURAUD
euraud_ticks = mt5.copy_ticks_from("EURAUD", datetime(2023,5,1,13), 1_000, mt5.COPY_TICKS_ALL)
euraud_ticks = pd.DataFrame(euraud_ticks)

# convert time in seconds into the datetime format
euraud_ticks['time']=pd.to_datetime(euraud_ticks['time'], unit='s')
euraud_ticks

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2023-05-01 16:00:00,1.65634,1.65641,0.0,0,1682956800151,134,0.0
1,2023-05-01 16:00:00,1.65631,1.65638,0.0,0,1682956800653,134,0.0
2,2023-05-01 16:00:00,1.65634,1.65641,0.0,0,1682956800748,134,0.0
3,2023-05-01 16:00:00,1.65631,1.65638,0.0,0,1682956800849,134,0.0
4,2023-05-01 16:00:00,1.65637,1.65644,0.0,0,1682956800960,134,0.0
...,...,...,...,...,...,...,...,...
995,2023-05-01 16:07:26,1.65572,1.65579,0.0,0,1682957246346,134,0.0
996,2023-05-01 16:07:27,1.65573,1.65580,0.0,0,1682957247056,134,0.0
997,2023-05-01 16:07:28,1.65574,1.65581,0.0,0,1682957248258,134,0.0
998,2023-05-01 16:07:28,1.65581,1.65588,0.0,0,1682957248356,134,0.0


### 3.3 - Request candles

In [7]:
# get bars from different symbols
eurgbp_rates = mt5.copy_rates_from_pos("EURGBP", mt5.TIMEFRAME_M5, 0, 1_000)
eurgbp_rates = pd.DataFrame(eurgbp_rates)

# convert time in seconds into the datetime format
eurgbp_rates['time']=pd.to_datetime(eurgbp_rates['time'], unit='s')
eurgbp_rates

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-05-16 12:30:00,0.86911,0.86927,0.86905,0.86920,356,1,0
1,2023-05-16 12:35:00,0.86920,0.86930,0.86897,0.86902,246,1,0
2,2023-05-16 12:40:00,0.86902,0.86919,0.86894,0.86918,219,1,0
3,2023-05-16 12:45:00,0.86919,0.86936,0.86913,0.86935,209,1,0
4,2023-05-16 12:50:00,0.86935,0.86955,0.86925,0.86927,243,1,0
...,...,...,...,...,...,...,...,...
995,2023-05-19 23:35:00,0.86826,0.86833,0.86818,0.86827,206,0,0
996,2023-05-19 23:40:00,0.86827,0.86838,0.86823,0.86835,148,0,0
997,2023-05-19 23:45:00,0.86835,0.86846,0.86830,0.86834,185,0,0
998,2023-05-19 23:50:00,0.86834,0.86836,0.86809,0.86811,251,0,0


## 4 - Send an order request

### 4.1 - Send the order

In [14]:
symbol = "USDJPY"
lot = 100
deviation = 20
magic = 707070

# Pending parameters
price = 139
expiration = datetime(2023, 5, 29)

trades = Trade(deviation=deviation, magic_number=magic)

send_success = trades.buy_stop(symbol=symbol, volume=lot, price=price, expiration=expiration)

Open pending order: Error 10018 - Market closed
Result: (Return Code) 10018 - (Comment) Market closed
Order type: BUY_STOP
Order ticket: 0
Symbol: USDJPY
Volume: 0.0
Price: 0.0
Bid: 0.0
Ask: 0.0


### 4.2 - Get the last result

In [9]:
trades.last_result

MqlTradeResult(retcode=<ENUM_TRADE_RETCODE.TRADE_RETCODE_MARKET_CLOSED: 10018>, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Market closed', request_id=4112016318, retcode_external=0, request=MqlTradeRequest(action=<ENUM_TRADE_REQUEST_ACTIONS.TRADE_ACTION_PENDING: 5>, symbol='USDJPY', magic=707070, order=0, volume=100.0, price=139.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=<ENUM_ORDER_TYPE.ORDER_TYPE_BUY_STOP: 4>, type_filling=<ENUM_ORDER_TYPE_FILLING.ORDER_FILLING_FOK: 0>, type_time=<ENUM_ORDER_TYPE_TIME.ORDER_TIME_SPECIFIED: 2>, expiration=datetime.datetime(2023, 5, 29, 0, 0, tzinfo=<UTC>), comment='', position=0, position_by=0))

In [10]:
a = mt5.orders_get()[0]._asdict()
a

{'ticket': 50459828331,
 'time_setup': 1684539842,
 'time_setup_msc': 1684539842112,
 'time_done': 0,
 'time_done_msc': 0,
 'time_expiration': 1685318400,
 'type': 4,
 'type_time': 2,
 'type_filling': 2,
 'state': 1,
 'magic': 707070,
 'position_id': 0,
 'position_by_id': 0,
 'reason': 3,
 'volume_initial': 100.0,
 'volume_current': 100.0,
 'price_open': 139.0,
 'sl': 0.0,
 'tp': 0.0,
 'price_current': 137.979,
 'price_stoplimit': 0.0,
 'symbol': 'USDJPY',
 'comment': '',
 'external_id': ''}

In [11]:
datetime.fromtimestamp(a["time_expiration"])

datetime.datetime(2023, 5, 28, 21, 0)

In [16]:
from datetime import timezone
import pytz
utc = pytz.timezone("UTC")
sp = pytz.timezone("America/Sao_Paulo")

a = datetime(2023, 5, 29, 0, 0)
a = utc.localize(a)

a = a.astimezone(sp)

a.timestamp()

1685318400.0